# Transformer Based Neural Network main run 


## Importing necessary libraries

In [ ]:
import time
import pandas as pd
import numpy as np
import os
from copy import deepcopy
import pyarrow
import random
from sklearn.preprocessing import StandardScaler
import torch


# 3rd party packages
#from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
#from torch.utils.tensorboard import SummaryWriter
from functools import partial
from running import UnsupervisedRunner
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## GPU check

In [ ]:
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

## Necessary Initialisations

In [ ]:
column_df = 4
list_temp = ['columns'] #,'actsoc_ep1_x_b3','ambientairtemperature','actpackvoltage_ep1_x_b3',
list_std = ['columns'] # list to give for standardization inside running, without the diff column
l2_reg = 1
lr = 0.0001
max_len = 1024
schedule_factor = 0.5
sch_patience = 3
mask_len = 5
mask_ratio = 0.2
folder_name = "results_"

## Folder creation for the necessary files to be stored

In [ ]:
os.makedirs(f"{folder_name}/eval_metrics")
os.makedirs(f"{folder_name}/eval_metrics/predplot_plotly")
writer = pd.ExcelWriter(f"{folder_name}/eval_metrics/{folder_name}.xlsx")

## Data Preprocessing

### Importing needed data

In [ ]:
parquet_file = r'path_to_import_data'

df_pd = pd.read_parquet(parquet_file,engine='pyarrow')
df_pd



### Slicing and dropping some values

In [ ]:
#df = df_pd.drop(columns=['time_abs'],axis=1)
list1 = ['all_required_cols']

list_col = ['columns_to_consider']

#list_temp = ['ambientairtemperature','actpackvoltage_ep1_x_b3','actcurrent_ep1_x_b3','actsoc_ep1_x_b3','time_diff']

df = df_pd[list_col]

df = df.rename(columns={'a': 'b'})
df = df[df['b'] < 600]
df = df.drop(columns=['b'],axis=1)


### Standardization data Import

In [ ]:
parquet_file1 = r'standardized_data'

filtered_df = pd.read_parquet(parquet_file1,engine='pyarrow')

parquet_file2 = r'original data'

org_df = pd.read_parquet(parquet_file2,engine='pyarrow')

list_t = ['columns to consider for prediction']

### Data split for train,val and test

In [ ]:
n_batch = list(df.index.unique())
random.shuffle(n_batch)
train = int(np.round(0.7*len(n_batch)))
val = train + int(np.round(0.2*len(n_batch)))
test = val + int(np.round(0.1*len(n_batch)))

train_df = df.loc[n_batch[:train]]
val_df = df.loc[n_batch[train+1:val]]
test_df = df.loc[n_batch[val+1:test]]


# PCA

In [ ]:
# pca = PCA(5)
# pca.fit(df)

In [ ]:
#te = df.groupby(df.index)
#a = te.size()
#print(a.idxmin())

In [ ]:
# #c_batch = df.pop('b')
# c_batch = pd.Series(df.pop('b'),dtype='int32')
# c_batch

In [ ]:
# exp_var_ratio = pca.explained_variance_ratio_

# plt.plot(exp_var_ratio)
# plt.xlabel('Number of components')
# plt.ylabel('Explained Variance Ratio')
# plt.show()

# cum_exp_var_ratio = np.cumsum(exp_var_ratio)

In [ ]:
# df_pca = pca.transform(df)

In [ ]:
# df_pca =  pd.DataFrame(df_pca)
# df_pca

In [ ]:
# merged_df = pd.concat([df_pd, df_pca], axis=1)
# merged_df

In [ ]:
# 'df_pca['b'] = c_batch
# #df_pca = df_pca.set_index('b')
# df_pca'

## Custom dataset function 

In [ ]:
class ImputationDataset(Dataset):
    """Dynamically computes missingness (noise) mask for each sample"""

    def __init__(self, df, indices, mean_mask_length=mask_len, masking_ratio=mask_ratio,  #mean_mask_len = 3
                 mode='separate', distribution='geometric', exclude_feats=None):
        super(ImputationDataset, self).__init__()

        self.data = df  # this is a subclass of the BaseData class in data.py
        self.IDs = indices  # list of data IDs, but also mapping between integer index and ID
        self.feature_df = self.data.loc[self.IDs]

        self.masking_ratio = masking_ratio
        self.mean_mask_length = mean_mask_length
        self.mode = mode
        self.distribution = distribution
        self.exclude_feats = exclude_feats

    def __getitem__(self, ind):
        """
        For a given integer index, returns the corresponding (seq_length, feat_dim) array and a noise mask of same shape
        Args:
            ind: integer index of sample in dataset
        Returns:
            X: (seq_length, feat_dim) tensor of the multivariate time series corresponding to a sample
            mask: (seq_length, feat_dim) boolean tensor: 0s mask and predict, 1s: unaffected input
            ID: ID of sample
        """

        X = self.feature_df.loc[self.IDs[ind]].values  # (seq_length, feat_dim) array # IDs[ind]
        mask = noise_mask(X, self.masking_ratio, self.mean_mask_length, self.mode, self.distribution,
                          self.exclude_feats)  # (seq_length, feat_dim) boolean array
        #print("aa",self.data.shape[0])
        return torch.from_numpy(X), torch.from_numpy(mask), self.IDs[ind]

    def update(self):
        self.mean_mask_length = min(20, self.mean_mask_length + 1)
        self.masking_ratio = min(1, self.masking_ratio + 0.05)

    def __len__(self):
        return len(self.IDs)

## Noise Mask Function

In [ ]:
def noise_mask(X, masking_ratio, lm=mask_len, mode='separate', distribution='geometric', exclude_feats=None):
    """
    Creates a random boolean mask of the same shape as X, with 0s at places where a feature should be masked.
    Args:
        X: (seq_length, feat_dim) numpy array of features corresponding to a single sample
        masking_ratio: proportion of seq_length to be masked. At each time step, will also be the proportion of
            feat_dim that will be masked on average
        lm: average length of masking subsequences (streaks of 0s). Used only when `distribution` is 'geometric'.
        mode: whether each variable should be masked separately ('separate'), or all variables at a certain positions
            should be masked concurrently ('concurrent')
        distribution: whether each mask sequence element is sampled independently at random, or whether
            sampling follows a markov chain (and thus is stateful), resulting in geometric distributions of
            masked squences of a desired mean length `lm`
        exclude_feats: iterable of indices corresponding to features to be excluded from masking (i.e. to remain all 1s)
    Returns:
        boolean numpy array with the same shape as X, with 0s at places where a feature should be masked
    """
    if exclude_feats is not None:
        exclude_feats = set(exclude_feats)

    if distribution == 'geometric':  # stateful (Markov chain)
        if mode == 'separate':  # each variable (feature) is independent
            mask = np.ones(X.shape, dtype=bool)
            #print(mask.shape)
            for m in range(X.shape[1]):  # feature dimension
                if exclude_feats is None or m not in exclude_feats:
                    mask[:, m] = geom_noise_mask_single(X.shape[0], lm, masking_ratio)  # time dimension
        

    return mask

In [ ]:
def geom_noise_mask_single(L, lm, masking_ratio):
    """
    Randomly create a boolean mask of length `L`, consisting of subsequences of average length lm, masking with 0s a `masking_ratio`
    proportion of the sequence L. The length of masking subsequences and intervals follow a geometric distribution.
    Args:
        L: length of mask and sequence to be masked
        lm: average length of masking subsequences (streaks of 0s)
        masking_ratio: proportion of L to be masked
    Returns:
        (L,) boolean numpy array intended to mask ('drop') with 0s a sequence of length L
    """
    keep_mask = np.ones(L, dtype=bool)
    p_m = 1 / lm  # probability of each masking sequence stopping. parameter of geometric distribution.
    p_u = p_m * masking_ratio / (1 - masking_ratio)  # probability of each unmasked sequence stopping. parameter of geometric distribution.
    p = [p_m, p_u]

    # Start in state 0 with masking_ratio probability
    state = int(np.random.rand() > masking_ratio)  # state 0 means masking, 1 means not masking
    for i in range(L):
        keep_mask[i] = state  # here it happens that state and masking value corresponding to state are identical
        if np.random.rand() < p[state]:
            state = 1 - state

    return keep_mask

## Loading the Model, Optimizer and Loss Modules

In [ ]:
from Model import TSTransformerEncoder
model = TSTransformerEncoder(column_df,max_len,['give_the_required_values_from_function'],pos_encoding='learnable',activation='gelu',norm='BatchNorm',freeze=False)

from loss import MaskedMSELoss
loss_module = MaskedMSELoss()

from torch.optim.optimizer import Optimizer 
from optimizer import RAdam
from torch.optim.lr_scheduler import ReduceLROnPlateau
optimizer = RAdam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=schedule_factor, patience=sch_patience, verbose=True)

In [ ]:
device = torch.device("cuda")
model.to(device)

## Collate Function

In [ ]:
def collate_unsuperv(data, max_len=None, mask_compensation=False):
    """Build mini-batch tensors from a list of (X, mask) tuples. Mask input. Create
    Args:
        data: len(batch_size) list of tuples (X, mask).
            - X: torch tensor of shape (seq_length, feat_dim); variable seq_length.
            - mask: boolean torch tensor of shape (seq_length, feat_dim); variable seq_length.
        max_len: global fixed sequence length. Used for architectures requiring fixed length input,
            where the batch length cannot vary dynamically. Longer sequences are clipped, shorter are padded with 0s
    Returns:
        X: (batch_size, padded_length, feat_dim) torch tensor of masked features (input)
        targets: (batch_size, padded_length, feat_dim) torch tensor of unmasked features (output)
        target_masks: (batch_size, padded_length, feat_dim) boolean torch tensor
            0 indicates masked values to be predicted, 1 indicates unaffected/"active" feature values
        padding_masks: (batch_size, padded_length) boolean tensor, 1 means keep vector at this position, 0 ignore (padding)
    """

    batch_size = len(data)
    features, masks,IDs = zip(*data)
    idx_dict = { '': { }, 
         '': { }}
    lengths = [X.shape[0] for X in features] 
    if max_len is None:
        max_len = max(lengths)
    X = torch.zeros(batch_size, max_len, features[0].shape[-1])  # (batch_size, padded_length, feat_dim)
    target_masks = torch.zeros_like(X,
                                    dtype=torch.bool)  # (batch_size, padded_length, feat_dim) masks related to objective
    idx_arr = []
    for i in range(batch_size):
        end = min(lengths[i], max_len)
        if lengths[i] > max_len:
            sl = random.randint(0,(lengths[i]-max_len))
            slen = min(lengths[i] - sl, max_len)
            X[i, :slen, :] = features[i][sl:sl+slen, :]
            target_masks[i, :slen, :] = masks[i][sl:sl+slen, :]
            idx = slen
            
        else:
            X[i,:end,:] = features[i][:end, :]
            target_masks[i, :end, :] = masks[i][:end, :]
            idx = 0
        
            
    targets = X.clone()
    X = X * target_masks  # mask input ##change
    padding_masks = padding_mask(torch.tensor(lengths, dtype=torch.float32), max_len=max_len)  # (batch_size, padded_length) boolean tensor, "1" means keep
    target_masks = ~target_masks  # inverse logic: 0 now means ignore, 1 means predict
    
    return X.to(device), targets.to(device), target_masks.to(device), padding_masks.to(device),IDs,idx

def padding_mask(lengths, max_len=None):
    """
    Used to mask padded positions: creates a (batch_size, max_len) boolean mask from a tensor of sequence lengths,
    where 1 means keep element at this position (time step)
    """
    batch_size = lengths.numel()
    max_len = max_len or lengths.max_val()  # trick works because of overloading of 'or' operator for non-boolean types
    return (torch.arange(0, max_len)
            .type_as(lengths)
            .repeat(batch_size, 1)
            .lt(lengths.unsqueeze(1)))

## Function to save model parameters

In [ ]:
def save_model(path, epoch, model, optimizer=None):
    if isinstance(model, torch.nn.DataParallel):
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()
    data = {'epoch': epoch,
            'state_dict': state_dict}
    if not (optimizer is None):
        data['optimizer'] = optimizer.state_dict()
    torch.save(data, path)

## Function to load the model

In [ ]:
def load_model(model, model_path, optimizer=None, resume=False, change_output=False,
               lr=None, lr_step=None, lr_factor=None):
    start_epoch = 0
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    state_dict = deepcopy(checkpoint['state_dict'])
    if change_output:
        for key, val in checkpoint['state_dict'].items():
            if key.startswith('output_layer'):
                state_dict.pop(key)
    model.load_state_dict(state_dict, strict=False)
    print('Loaded model from {}. Epoch: {}'.format(model_path, checkpoint['epoch']))

    # resume optimizer parameters
    if optimizer is not None and resume:
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
            start_epoch = checkpoint['epoch']
            start_lr = lr
            for i in range(len(lr_step)):
                if start_epoch >= lr_step[i]:
                    start_lr *= lr_factor[i]
            for param_group in optimizer.param_groups:
                param_group['lr'] = start_lr
            print('Resumed optimizer with start lr', start_lr)
        else:
            print('No optimizer parameters in checkpoint.')
    if optimizer is not None:
        return model, optimizer, start_epoch
    else:
        return model

## Training initialisation

In [ ]:
def pipeline_factory():
    """For the task specified in the configuration returns the corresponding combination of
    Dataset class, collate function and Runner class."""
    
    return ImputationDataset, collate_unsuperv, UnsupervisedRunner

dataset_class, collate_fn, runner_class = pipeline_factory()

In [ ]:
a = train_df.index.unique()

train_dataset = dataset_class(train_df, a,mask_len,mask_ratio)

train_loader = DataLoader(train_dataset,
                          batch_size=your_wish, 
                          shuffle=True,
                          collate_fn=lambda x: collate_unsuperv(x, max_len=model.max_len))

In [ ]:
b = val_df.index.unique()

val_dataset = dataset_class(val_df, b,mask_len,mask_ratio)

val_loader = DataLoader(val_dataset, 
                         batch_size=your_wish,
                          shuffle=True,
                          collate_fn=lambda x: collate_unsuperv(x, max_len=model.max_len)) #

In [ ]:
device = torch.device("cuda")
trainer = UnsupervisedRunner(model, train_loader, device, loss_module,org_df,folder_name,list_std,optimizer,l2_reg=l2_reg)
validator = UnsupervisedRunner(model, val_loader, device, loss_module,org_df,folder_name,list_std,optimizer,scheduler,l2_reg=l2_reg)

## Training Run

In [ ]:
%%time
no_of_epochs = 'for_iterations'
pt_epoch = []
pt_loss = []
vl_loss = []
patience = 10
best_val_loss = float('inf')
counter = 0

vl_loss_best = 'threshold'
best_epoch = -1

dict_pt = {}
if not os.path.exists(f"path_for_training_models"):
    os.makedirs(f"path_for_training_models")
epoch_time = []
path = f"path_for_training_models"
for epochs in range(no_of_epochs):
    start = time.time()
    a,b,c = trainer.train_epoch(epochs)
    d,e = validator.evaluate(epochs)
    #if (epochs % 10 == 0):
    #if b < 10:5
    #dict_pt[f"epoch_{epochs}"] = c
    vl_loss.append(e)
    pt_epoch.append(a)
    pt_loss.append(b)
    
    if e < best_val_loss:
        best_val_loss = e
        counter = 0
        save_model(path+'epoch_{}'.format(epochs), epochs, model, optimizer)
    else:
        counter += 1
    #print(best_val_loss)
    # Stop training if the validation loss has not improved for a certain number of epochs
    if counter >= patience:
        print(f'Early stopping after {epochs} epochs')
        end = time.time()
        epoch_t = end - start
        epoch_time.append(epoch_t)
        break
    
#     if vl_loss[epochs] < vl_thres:
#         vl_loss_best = vl_loss[epochs]
#         best_epoch = epochs 
    
    #save_model(path+'epoch_{}'.format(epochs), epochs, model, optimizer)
    end = time.time()
    epoch_t = end - start
    epoch_time.append(epoch_t)

### Saving the minimum training and validation losses epochs

In [ ]:
epoch_val_pt = pt_loss.index(min(pt_loss))
epoch_valid_pt = vl_loss.index(min(vl_loss))

print(f"Minimum Training loss at {epoch_val_pt} epoch \n Minimum Validation loss at {epoch_valid_pt} epoch")

### Saving the metrics to an excel

In [ ]:
epoch_df = pd.DataFrame({"epochs":pt_epoch,"validation_loss":vl_loss,"training_loss":pt_loss,"time_for_epoch":epoch_time})
epoch_df.to_excel(writer, sheet_name='tr_val_metrics', index=False)

### Printing the epoch of the best model

In [ ]:
epoch_valid_pt

### Training and validation loss curve

In [ ]:
import matplotlib.pyplot as plt

plt.suptitle('Training and valiation loss per epoch', fontsize=15)
plt.xlabel('epochs', fontsize=10, rotation='horizontal')
#plt.ylim([0,0.1])
plt.plot(pt_epoch,pt_loss)
plt.plot(pt_epoch,vl_loss)
plt.legend(["Training loss", "Validation loss"], loc ="upper right")
plt.savefig(f"/mnt/proj/emob-da1/zz_thesis_work/Thesis - Copy/{folder_name}/eval_metrics/train_val_plot.png")
plt.show()

### Getting the best model from the training run and loading the model

In [ ]:
model, optimizer, start_epoch = load_model(model, model_path =f"path_to_save_best_epoch_model",   #{epoch_valid_pt}
                                           optimizer = optimizer ,
                                           resume=False ,
                                           change_output=False ,
                                           lr=1e-3)

## Testing Initialisation

In [ ]:
c = test_df.index.unique()

test_dataset = dataset_class(test_df, c,mask_len,mask_ratio)

test_loader = DataLoader(test_dataset, 
                          batch_size= 'your_wish', 
                          shuffle=False,
                          collate_fn=lambda x: collate_unsuperv(x, max_len=model.max_len)) #

In [ ]:
l = ['the_cols_you_want']
device = torch.device("cuda")
tester = UnsupervisedRunner(model, test_loader, device, loss_module,org_df,folder_name,list_std,optimizer,l2_reg=l2_reg)

## Test Run

In [ ]:
no_of_epochs = 1
test_epoch = []
test_loss = []
epoch_time = []
rmse_dict = dict()
mae_dict = dict()
for epochs in range(no_of_epochs):
    start = time.time()
    temp_a,b,c,d = tester.test_epoch(epochs)
    test_epoch.append(temp_a)
    test_loss.append(b)
    
    rmse_dict[f"epoch_path"] = c
    mae_dict[f"epoch_path"] = d
    
    end = time.time()
    epoch_t = end - start
    epoch_time.append(epoch_t)

### Saving the test losses, rmse and mae metrics to an excel file

In [ ]:
dd = pd.DataFrame(rmse_dict)
dd['idx'] = test_df.index.unique()
dd = dd.set_index('idx')
dd['batch_rmse'] = dd.mean(axis=1)

ee = pd.DataFrame(mae_dict)
ee['idx'] = test_df.index.unique()
ee = ee.set_index('idx')
ee['batch_mae'] = ee.mean(axis=1)

test_ex = pd.DataFrame({"epochs":test_epoch,"test_loss":test_loss,"time_for_epoch":epoch_time})
test_ex.to_excel(writer, sheet_name='test_metrics', index=False)

dd.to_excel(writer, sheet_name='rmse', index=False)
ee.to_excel(writer, sheet_name='mae', index=False)

writer.save()

### Test loss curve

In [ ]:
import matplotlib.pyplot as plt

plt.suptitle('Test loss per epoch', fontsize=15)
plt.xlabel('epochs', fontsize=10, rotation='horizontal')
#plt.ylim([0,0.1])
plt.plot(test_epoch,test_loss)
plt.savefig(f"/path_to_save_file")
plt.show()

### Visualization code

In [ ]:
df_org = the original file
df_pred = the predicted file

fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15, 8))

columns = ['ambientairtemperature','actsoc_ep1_x_b3','actpackvoltage_ep1_x_b3',
'actcurrent_ep1_x_b3'
]    #         

df_vstack_org = df_org.iloc[:100]
df_vstack_pred = df_pred.iloc[:100]

# diff = df_vstack_org - df_vstack_pred

# mask = diff > 0

for i in range(4):
    min_org = min(df_vstack_org[columns[i]])
    min_pred = min(df_vstack_pred[columns[i]])
    max_org = max(df_vstack_org[columns[i]])
    max_pred = max(df_vstack_pred[columns[i]])
    
    fin_min = min(min_org,min_pred)
    fin_max = max(max_org,max_pred)
    for j in range(2):
# Plot each feature vertically
        if j == 0:
            axes[i,j].plot(df_vstack_org.index, df_vstack_org[columns[i]],color = 'grey', label = 'aa')
            
            axes[i,j].scatter(df_vstack_org[columns[i]][df_mask[columns[i]]].index, df_vstack_org[columns[i]][df_mask[columns[i]]],color = 'blue', label = 'bb')
            axes[i,j].set_ylabel(f"{columns[i]}")
            axes[i,j].set_title(f"Original plot of {columns[i]}")
            axes[i,j].set_ylim(fin_min,fin_max)
            
        if j == 1:
            axes[i,j].plot(df_vstack_pred.index, df_vstack_pred[columns[i]],color = 'grey', label = 'aa')
            axes[i,j].scatter(df_vstack_pred[columns[i]][df_mask[columns[i]]].index, df_vstack_pred[columns[i]][df_mask[columns[i]]],color = 'green', label = 'aa')
            axes[i,j].set_ylabel(f"{columns[i]}")
            axes[i,j].set_title(f"Predicted plot of {columns[i]}")
            axes[i,j].set_ylim(fin_min,fin_max)
        
        axes[-1,j].set_xlabel('Time')

# Set common x-axis label


# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()